# Variable Audit Tool

This notebook provides an interactive tool to audit variable changes within a process instance.

## Features
- Select a process definition from a dropdown
- Choose a specific process instance (shows business key, start time, and status)
- Select one or more variables to audit
- View a formatted table showing all variable updates with timestamps and activity context

## Usage
1. Run all cells in order
2. Use the dropdowns to select a process and instance
3. Select variables to audit (hold Ctrl/Cmd for multiple)
4. Click "Show Variable History" to see the audit trail

In [ ]:
# Audit Variable Events - Interactive Notebook
# This notebook allows you to audit variable changes for a process instance

import operaton
from operaton import Operaton
from datetime import datetime, timedelta

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Initialize environment (loads API configuration from localStorage)
await operaton.load_env()

# Load BPMN moddle for parsing process definitions
await operaton.load_bpmn_moddle()

In [ ]:
# Helper function to convert timestamps
def to_dt(value):
    """Convert Camunda timestamp to datetime with timezone adjustment."""
    return datetime.strptime(value, "%Y-%m-%dT%H:%M:%S.%f%z") + timedelta(hours=2)

# Fetch all process definitions (latest versions only)
all_definitions = Operaton.get('/process-definition?latestVersion=true')
definitions_by_key = {d['key']: d for d in all_definitions}
print(f"Found {len(all_definitions)} process definitions")

In [ ]:
# State management
state = {
    'process_definition': None,
    'process_instances': [],
    'selected_instance': None,
    'variable_details': [],
    'available_variables': [],
    'activity_by_id': {}
}

# Create widgets
process_dropdown = widgets.Dropdown(
    options=[('-- Select Process Definition --', None)] + [(f"{d['name'] or d['key']} (v{d['version']})", d['key']) for d in sorted(all_definitions, key=lambda x: x['name'] or x['key'])],
    value=None,
    description='Process:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

instance_dropdown = widgets.Dropdown(
    options=[('-- Select Process Instance --', None)],
    value=None,
    description='Instance:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px'),
    disabled=True
)

variable_select = widgets.SelectMultiple(
    options=[],
    description='Variables:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px', height='150px'),
    disabled=True
)

audit_button = widgets.Button(
    description='Show Variable History',
    button_style='primary',
    disabled=True,
    layout=widgets.Layout(width='200px')
)

output_area = widgets.Output()

In [ ]:
# Event handlers
def on_process_change(change):
    """Handle process definition selection."""
    if change['new'] is None:
        instance_dropdown.options = [('-- Select Process Instance --', None)]
        instance_dropdown.disabled = True
        variable_select.options = []
        variable_select.disabled = True
        audit_button.disabled = True
        return
    
    process_key = change['new']
    state['process_definition'] = definitions_by_key.get(process_key)
    
    with output_area:
        clear_output()
        print(f"Loading instances for {process_key}...")
    
    # Fetch process instances (history for completed ones too)
    try:
        instances = Operaton.get(f'/history/process-instance?processDefinitionKey={process_key}&maxResults=100&sortBy=startTime&sortOrder=desc')
        state['process_instances'] = instances
        
        # Update instance dropdown
        instance_options = [('-- Select Process Instance --', None)]
        for inst in instances:
            business_key = inst.get('businessKey', '')
            start_time = to_dt(inst['startTime']).strftime('%Y-%m-%d %H:%M') if inst.get('startTime') else ''
            status = '✓' if inst.get('endTime') else '⏳'
            label = f"{status} {business_key or inst['id'][:8]} ({start_time})"
            instance_options.append((label, inst['id']))
        
        instance_dropdown.options = instance_options
        instance_dropdown.disabled = False
        
        with output_area:
            clear_output()
            print(f"Found {len(instances)} instances")
    except Exception as e:
        with output_area:
            clear_output()
            print(f"Error loading instances: {e}")

def on_instance_change(change):
    """Handle process instance selection."""
    if change['new'] is None:
        variable_select.options = []
        variable_select.disabled = True
        audit_button.disabled = True
        return
    
    instance_id = change['new']
    state['selected_instance'] = instance_id
    
    with output_area:
        clear_output()
        print(f"Loading variable history for instance {instance_id}...")
    
    try:
        # Fetch variable history details
        detail = Operaton.get(f'/history/detail?processInstanceId={instance_id}&variableUpdates=true')
        state['variable_details'] = detail
        
        # Get unique variable names
        variable_names = sorted(set(d.get('variableName', '') for d in detail if d.get('variableName')))
        state['available_variables'] = variable_names
        
        variable_select.options = variable_names
        variable_select.disabled = False
        audit_button.disabled = False
        
        # Also load BPMN model for activity names
        if state['process_definition']:
            try:
                xml_response = Operaton.get(f"/process-definition/{state['process_definition']['id']}/xml")
                xml = xml_response['bpmn20Xml']
                import js
                moddle = js.createBpmnModdle()
                # We need to parse async but we're in a sync callback
                # Store XML for later parsing
                state['bpmn_xml'] = xml
            except Exception as e:
                print(f"Warning: Could not load BPMN model: {e}")
        
        with output_area:
            clear_output()
            print(f"Found {len(detail)} variable updates across {len(variable_names)} variables")
            print(f"Select variables to audit and click 'Show Variable History'")
    except Exception as e:
        with output_area:
            clear_output()
            print(f"Error loading variable details: {e}")

def on_audit_click(button):
    """Display the variable audit table."""
    selected_vars = list(variable_select.value)
    if not selected_vars:
        with output_area:
            clear_output()
            print("Please select at least one variable to audit")
        return
    
    detail = state['variable_details']
    activity_by_id = state.get('activity_by_id', {})
    
    # Filter and sort the details
    filtered = [x for x in detail if x.get('variableName') in selected_vars]
    sorted_details = sorted(filtered, key=lambda x: x['time'])
    
    with output_area:
        clear_output()
        
        html = []
        html.append('<style>')
        html.append('table.audit { border-collapse: collapse; width: 100%; }')
        html.append('table.audit th, table.audit td { border: 1px solid #ddd; padding: 8px; text-align: left; }')
        html.append('table.audit tr:nth-child(even) { background-color: #f9f9f9; }')
        html.append('table.audit th { background-color: #4CAF50; color: white; }')
        html.append('</style>')
        html.append('<table class="audit">')
        html.append('<tr><th>Type</th><th>Variable</th><th>Value</th><th>Activity</th><th>Time</th></tr>')
        
        for d in sorted_details:
            var_type = d.get('type', '')
            var_name = d.get('variableName', '')
            value = d.get('value', '')
            # Truncate long values
            if isinstance(value, str) and len(value) > 100:
                value = value[:100] + '...'
            
            # Try to get activity name
            activity_instance_id = d.get('activityInstanceId', '')
            activity_name = ''
            if activity_instance_id and ':' in activity_instance_id:
                activity_id = activity_instance_id.split(':')[0]
                activity_name = activity_by_id.get(activity_id, {}).get('name', activity_id)
            
            time_str = to_dt(d['time']).strftime('%Y-%m-%d %H:%M:%S') if d.get('time') else ''
            
            html.append('<tr>')
            html.append(f'<td>{var_type}</td>')
            html.append(f'<td>{var_name}</td>')
            html.append(f'<td>{value}</td>')
            html.append(f'<td>{activity_name}</td>')
            html.append(f'<td>{time_str}</td>')
            html.append('</tr>')
        
        html.append('</table>')
        html.append(f'<p><em>Showing {len(sorted_details)} variable updates</em></p>')
        display(HTML("".join(html)))

# Attach event handlers
process_dropdown.observe(on_process_change, names='value')
instance_dropdown.observe(on_instance_change, names='value')
audit_button.on_click(on_audit_click)

In [ ]:
# Display the interactive interface
display(widgets.VBox([
    widgets.HTML('<h2>🔍 Variable Audit Tool</h2>'),
    widgets.HTML('<p>Select a process definition, then an instance, and choose which variables to audit.</p>'),
    process_dropdown,
    instance_dropdown,
    widgets.HTML('<p><em>Hold Ctrl/Cmd to select multiple variables:</em></p>'),
    variable_select,
    audit_button,
    output_area
]))